Lettura DataSet

In [1]:
import warnings
import torch
import pandas as pd
import utils as u
import os
from loader import load_data, data_to_pyg
from train import train, test
from models import models
from argparse import ArgumentParser
from models.custom_gat.model import GAT



df_classes = pd.read_csv("elliptic_txs_classes.csv")
df_edgelist = pd.read_csv("elliptic_txs_edgelist.csv")
df_features = pd.read_csv("elliptic_txs_features.csv",header=None)

df_features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203769 entries, 0 to 203768
Columns: 167 entries, 0 to 166
dtypes: float64(165), int64(2)
memory usage: 259.6 MB


In [2]:

""" df_classes = df_classes.loc[df_classes.iloc[:, 1] != 'unknown']

nodi_da_mantenere = set(df_classes['txId'])

df_edgelist = df_edgelist[df_edgelist['txId1'].isin(nodi_da_mantenere) & df_edgelist['txId2'].isin(nodi_da_mantenere)]

df_edgelist.to_csv('data/elliptic_txs_edgelist.csv', index=False)

istanze_colonna = df_classes['class'].value_counts()

print(istanze_colonna) """



" df_classes = df_classes.loc[df_classes.iloc[:, 1] != 'unknown']\n\nnodi_da_mantenere = set(df_classes['txId'])\n\ndf_edgelist = df_edgelist[df_edgelist['txId1'].isin(nodi_da_mantenere) & df_edgelist['txId2'].isin(nodi_da_mantenere)]\n\ndf_edgelist.to_csv('data/elliptic_txs_edgelist.csv', index=False)\n\nistanze_colonna = df_classes['class'].value_counts()\n\nprint(istanze_colonna) "

Rete 

In [3]:
import networkx as nx

import matplotlib.pyplot as plt  # Solo se desideri visualizzare il grafico


# Crea un grafo non diretto da edgelist
G = nx.from_pandas_edgelist(df_edgelist, 'txId1', 'txId2')

print(G)



Graph with 203769 nodes and 234355 edges


In [4]:
import networkx as nx

# MISURE DI CENTRALITA'

# Calcola la centralità di grado per ogni nodo
degree_centrality = nx.degree_centrality(G)

# Calcola la closeness centrality per ogni nodo
closeness_centrality = nx.closeness_centrality(G)

# Calcola la betweenness centrality per ogni nodo
betweenness_centrality = nx.betweenness_centrality(G)

# Calcola la eigenvector centrality per ogni nodo
eigenvector_centrality = nx.eigenvector_centrality(G)

# Calcola la pagerank centrality per ogni nodo
pagerank_centrality = nx.pagerank(G)

# Calcola la harmonic centrality per ogni nodo
harmonic_centrality = nx.harmonic_centrality(G)

# MISURE DI CLUSTERING

# Calcola il clustering coefficient per ogni nodo
clustering_coefficient = nx.clustering(G)

# Calcola il clustering coefficient per ogni nodo
local_clustering_coefficient = nx.clustering(G)




# Aggiungi il clustering coefficient come feature ai nodi

nx.set_node_attributes(G, clustering_coefficient, 'clustering_coefficient')
nx.set_node_attributes(G, local_clustering_coefficient, 'local_clustering_coefficient')
nx.set_node_attributes(G, degree_centrality, 'degree_centrality')
nx.set_node_attributes(G, closeness_centrality, 'closeness_centrality')
nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')
nx.set_node_attributes(G, eigenvector_centrality, 'eigenvector_centrality')
nx.set_node_attributes(G, pagerank_centrality, 'pagerank_centrality')
nx.set_node_attributes(G, harmonic_centrality, 'harmonic_centrality')



In [13]:
import pandas as pd
import networkx as nx

# Supponiamo che tu abbia già creato il tuo grafo G

# Estrai le features dai nodi
node_data = {'txId': list(G.nodes),
             'clustering_coefficient': list(nx.get_node_attributes(G, 'clustering_coefficient').values()),
             'degree_centrality': list(nx.get_node_attributes(G, 'degree_centrality').values()),
             'closeness_centrality': list(nx.get_node_attributes(G, 'closeness_centrality').values()),
             'betweenness_centrality': list(nx.get_node_attributes(G, 'betweenness_centrality').values()),
             'eigenvector_centrality': list(nx.get_node_attributes(G, 'eigenvector_centrality').values()),
             'local_clustering_coefficient': list(nx.get_node_attributes(G, 'local_clustering_coefficient').values()),
             'pagerank_centrality': list(nx.get_node_attributes(G, 'pagerank_centrality').values()),
             'harmonic_centrality': list(nx.get_node_attributes(G, 'harmonic_centrality').values())
             }

# Crea il DataFrame pandas
df_features_graph = pd.DataFrame(node_data)

# Converti la colonna 'node_id' in int64
df_features_graph['txId'] = df_features_graph['txId'].astype('int64')

# Converti tutte le colonne numeriche tranne 'node_id' in float64
numerical_columns = df_features_graph.columns.difference(['txId'])
df_features_graph[numerical_columns] = df_features_graph[numerical_columns].astype('float64')

# Salva il DataFrame nella cartella corrente in formato CSV
df_features_graph.to_csv('data/elliptic_txs_features.csv', index=False)

    
""" df_classes = df_classes[df_classes['txId'].isin(nodi_da_mantenere)]

txId_comuni = df_features_graph['txId'].isin(df_classes['txId'])

df_classes = df_classes[df_classes['txId'].isin(df_features_graph['txId'])]

df_classes = df_classes.reset_index(drop=True)

"""

df_classes.to_csv('data/elliptic_txs_classes.csv', index=False) 
df_edgelist.to_csv('data/elliptic_txs_edgelist.csv', index=False)
